### Use Qiskit to run Bell-state circuit with ideal and noisy simu as if IonQ Harmony
Qiskit intro https://ionq.com/docs/get-started-with-qiskit 
All jobs are reported : https://cloud.ionq.com/jobs 

In [36]:
from pprint import pprint
from time import time
from qiskit import QuantumCircuit
from qiskit_ionq import IonQProvider
provider = IonQProvider()   # Remember to set env IONQ_API_KEY='....'
print(provider.backends())  # Show all backends

[<IonQSimulatorBackend('ionq_simulator')>, <IonQQPUBackend('ionq_qpu')>]


In [37]:
# Get IonQ's simulator backend to be used for both: ideal and noisy simulations
backend = provider.get_backend("ionq_simulator")

In [38]:
shots=10000
# Create a basic Bell State circuit:
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])
print(qc)
# Run the circuit on IonQ's platform:
job = backend.run(qc, shots=shots) 

# Print the counts
print(type(job),'counts:',job.get_counts())

     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 
<class 'qiskit_ionq.ionq_job.IonQJob'> counts: {'00': 5069, '11': 4931}


In [39]:
# Setup noise model as one of:
simList=['ideal', 'harmony-2', 'aria-1', 'aria-2', 'forte-1']

In [40]:
for noiseN in simList[:2]:
    #noiseN='harmony-1'
    print('simu IonQ:%s ....'%noiseN)
    job = backend.run(qc, shots=shots, noise_model=noiseN)
    print('simu noise=%s, counts:'%noiseN,job.get_counts())

simu IonQ:ideal ....
simu noise=ideal, counts: {'00': 4887, '11': 5113}
simu IonQ:harmony-2 ....
simu noise=harmony-2, counts: {'00': 4687, '01': 310, '10': 292, '11': 4711}
simu IonQ:aria-1 ....
simu noise=aria-1, counts: {'00': 5010, '01': 28, '10': 28, '11': 4934}
simu IonQ:aria-2 ....
simu noise=aria-2, counts: {'00': 4941, '01': 56, '10': 53, '11': 4950}
simu IonQ:forte-1 ....
simu noise=forte-1, counts: {'00': 4976, '01': 9, '10': 20, '11': 4995}


In [42]:
cal=backend.calibration()
jid=job.job_id()
print(cal,jid)

None dc0c2c31-3734-4d38-aa38-798e1f6d1e56


In [43]:
# construct highly parallele bell-circuit
nPar=3 
n2=2
# Create a basic Bell State circuit:
qc = QuantumCircuit(n2*nPar)
for j in range(nPar):
    i0=j*n2; i1=i0+1
    qc.h(i0)
    qc.cx(i0, i1)
qc.measure_all()
print(qc)
# Run the circuit on IonQ's platform:
job = backend.run(qc, shots=shots) 

# Print the counts
print(type(job),'counts:'); pprint(job.get_counts())


        ┌───┐      ░ ┌─┐               
   q_0: ┤ H ├──■───░─┤M├───────────────
        └───┘┌─┴─┐ ░ └╥┘┌─┐            
   q_1: ─────┤ X ├─░──╫─┤M├────────────
        ┌───┐└───┘ ░  ║ └╥┘┌─┐         
   q_2: ┤ H ├──■───░──╫──╫─┤M├─────────
        └───┘┌─┴─┐ ░  ║  ║ └╥┘┌─┐      
   q_3: ─────┤ X ├─░──╫──╫──╫─┤M├──────
        ┌───┐└───┘ ░  ║  ║  ║ └╥┘┌─┐   
   q_4: ┤ H ├──■───░──╫──╫──╫──╫─┤M├───
        └───┘┌─┴─┐ ░  ║  ║  ║  ║ └╥┘┌─┐
   q_5: ─────┤ X ├─░──╫──╫──╫──╫──╫─┤M├
             └───┘ ░  ║  ║  ║  ║  ║ └╥┘
meas: 6/══════════════╩══╩══╩══╩══╩══╩═
                      0  1  2  3  4  5 
<class 'qiskit_ionq.ionq_job.IonQJob'> counts:
{'000000': 1239,
 '000011': 1190,
 '001100': 1314,
 '001111': 1227,
 '110000': 1307,
 '110011': 1284,
 '111100': 1222,
 '111111': 1217}


In [44]:
import random
for noiseN in simList:
    #noiseN='harmony-1'
    print('\nIonQ simu: %s nQ=%d ....'%(noiseN,qc.num_qubits),end='')
    T0=time()
    try:
        job = backend.run(qc, shots=shots, noise_model=noiseN)
        counts=job.get_counts()
    except:
        print("Failed"); continue
    
    elaT=time()-T0
    # Find the bitstring with the maximum count
    max_bitstring = max(counts, key=counts.get)
    print('elaT=%.1f sec, num bistrings=%d,  max mshot=%d for %s'%(elaT,len(counts),counts[max_bitstring],max_bitstring))
    random_samples = random.sample(list(counts.items()), 5)
    for bitstring, count in random_samples: # Print the selected random samples
        print("   %s:%d,"%(count,bitstring),end='')
    print()
   


IonQ simu: ideal nQ=6 ....elaT=6.5 sec, num bistrings=8,  max mshot=1303 for 111100


TypeError: %d format: a real number is required, not str

In [ ]:
from qiskit import  transpile
qcT = transpile(qc, backend=backend, optimization_level=3, seed_transpiler=42)
print(qcT)